# A* 3x3 Puzzle Solver

This project incorporates the A* search algorithm to solve the famous moving tile puzzle.

![puzzle](https://miro.medium.com/max/1400/1*OjlGSVqTDrBOMiIR0-Bx9Q.png)

We will be solving the exact puzzle above.

### Import neccessary packages

In [5]:
import heapq
import copy
import time

### Heuristic Value
An important distinction from the A* algorithm versus other search algorithm is that it uses a heuristic value. This value helps the algorithm "know" how close it is to the solution. A good heuristic value is one that is large but is always less than or equal to the number of moves it takes from one state to the end state. In this project, we will use the l1 norm (Manhattan Distance) of every tile to where it is supposed to be. 

In [7]:
def get_manhattan_distance(from_state, to_state=[1, 2, 3, 4, 5, 6, 7, 8, 0]):
    
    goaldict = {}
    for idx in range(len(to_state)):
        if to_state[idx] != 0:
            goaldict[to_state[idx]] = [idx//3, idx%3]
    currentdict = {}
    for idx in range(len(from_state)):
        if from_state[idx] != 0:
            currentdict[from_state[idx]] = [idx//3, idx%3]
    distance = 0
    for num in currentdict.keys():
        ccoord = currentdict[num]
        newcoord = goaldict[num]
        distance += abs(ccoord[0]-newcoord[0])+abs(ccoord[1]-newcoord[1]) 
    return distance

In [14]:
# Test with initial puzzle:
get_manhattan_distance([4, 1, 6, 0, 3, 5, 2, 7, 8])

11

### Successive States
Now lets make a couple of functions to get and print (in order) the successive states of a given state along with their heuristic values.

In [9]:
def get_succ(state):
    
    movesdict = {
    0: [1, 3],
    1: [0, 2, 4], 
    2: [1, 5],
    3: [0, 4, 6], 
    4: [1, 3, 5, 7], 
    5: [2, 4, 8], 
    6: [3, 7], 
    7: [4, 6, 8], 
    8: [5, 7]
    }
    succ_states = []
    moves = movesdict[state.index(0)]
    for move in moves:
        state1 = copy.copy(state)    
        state1[state.index(0)] = state1[move]
        state1[move] = 0
        succ_states.append(state1)
    return sorted(succ_states)

def print_succ(state):
    
    succ_states = get_succ(state)
    for succ_state in succ_states:
        print(succ_state, "h={}".format(get_manhattan_distance(succ_state)))

In [15]:
# Test with initial puzzle:
print_succ([4, 1, 6, 0, 3, 5, 2, 7, 8])

[0, 1, 6, 4, 3, 5, 2, 7, 8] h=10
[4, 1, 6, 2, 3, 5, 0, 7, 8] h=10
[4, 1, 6, 3, 0, 5, 2, 7, 8] h=12


### Implement A*
Finally lets implement the algorithm given the helper functions we just created.
NOTE: We use a heapque to order our queue by 
        
        f(n) = heuristic value (h) + # of moves taken (g)

In [17]:
path = []

#Recursive Path Finder:
def pathfinder(nd, end):
    
    path.append(end)
    if nd[str(end)]["parent"] == -1:
        return path
    else:
        return pathfinder(nd, nd[str(end)]["parent"])


#Final Solver for Given State
def solve(state, goal_state=[1, 2, 3, 4, 5, 6, 7, 8, 0]):
    
    t1 = time.time()
    pq = []
    closed = []
    g = 0
    visited = []
    openl = []
    # 1. Place the starting node into OPEN and find its f (n) value.
    h = get_manhattan_distance(state)
    heapq.heappush(pq, (g+h, state, (g, h, -1)))
    # 2. Remove the node from pq, having the smallest f (n) value. If it is a goal node, then stop and return to success.
    while len(pq) != 0:
        currnode = heapq.heappop(pq)
        nodestate = currnode[1]
        closed.append(currnode)
        visited.append(nodestate)
        if nodestate == goal_state:
            break
    # 3. Else remove the node from pq, and find all its successors.
        else:
            g = currnode[2][0]
            for child in get_succ(nodestate):
                if child not in visited and child not in openl:
    # 4. Find the f (n) value of all the successors, place them into pq, and place the removed node into CLOSE.
                    h = get_manhattan_distance(child)
                    heapq.heappush(pq, (g+h+1, child, (g+1, h, nodestate)))
                    openl.append(child)
            #Below 2 comments used to debug 
            #print(*pq, sep='\n')
            #print("________________")
    if len(pq) == 0 and nodestate != goal_state:
        print("FAILURE")
    nodedict = {}
    for node in closed:
        nodedict[str(node[1])] = {
            "h": node[2][1], 
            "moves": node[2][0], 
            "parent": node[2][2]
        }
    #print(nodedict)
    path =  []
    output = pathfinder(nodedict, goal_state)
    mmoves = nodedict[str(output[0])]["moves"]
    output1 = output[-mmoves-1:]
    for node in output1[::-1]:
        print(str(node)+" h="+str(nodedict[str(node)]["h"])+" moves: "+str(nodedict[str(node)]["moves"]))
    t2 = time.time()
    print("Time to solve: "+str(round((t2-t1)*1000))+" milliseconds")

## Now lets solve the puzzle!

In [18]:
solve([4, 1, 6, 0, 3, 5, 2, 7, 8])

[4, 1, 6, 0, 3, 5, 2, 7, 8] h=11 moves: 0
[0, 1, 6, 4, 3, 5, 2, 7, 8] h=10 moves: 1
[1, 0, 6, 4, 3, 5, 2, 7, 8] h=9 moves: 2
[1, 3, 6, 4, 0, 5, 2, 7, 8] h=8 moves: 3
[1, 3, 6, 4, 5, 0, 2, 7, 8] h=7 moves: 4
[1, 3, 0, 4, 5, 6, 2, 7, 8] h=6 moves: 5
[1, 0, 3, 4, 5, 6, 2, 7, 8] h=5 moves: 6
[0, 1, 3, 4, 5, 6, 2, 7, 8] h=6 moves: 7
[4, 1, 3, 0, 5, 6, 2, 7, 8] h=7 moves: 8
[4, 1, 3, 2, 5, 6, 0, 7, 8] h=6 moves: 9
[4, 1, 3, 2, 5, 6, 7, 0, 8] h=5 moves: 10
[4, 1, 3, 2, 0, 6, 7, 5, 8] h=6 moves: 11
[4, 1, 3, 0, 2, 6, 7, 5, 8] h=5 moves: 12
[0, 1, 3, 4, 2, 6, 7, 5, 8] h=4 moves: 13
[1, 0, 3, 4, 2, 6, 7, 5, 8] h=3 moves: 14
[1, 2, 3, 4, 0, 6, 7, 5, 8] h=2 moves: 15
[1, 2, 3, 4, 5, 6, 7, 0, 8] h=1 moves: 16
[1, 2, 3, 4, 5, 6, 7, 8, 0] h=0 moves: 17
Time to solve: 17 milliseconds


## Conclusion
The A* algorithm seems to have found the shortest path to the solution state and it only took 17 moves and was solved very quickly. If we had used a different search algorithm which was uninformed, BFS for example, it would've taken much longer to reach the solution as we'd be "shooting in the dark". In conclusion, the A* search algorithm is a useful tool when we have a specific goal state as well as a good heuristic function to speed up our search. 